# Import libraries and initial setup

In [9]:
import os
from pathlib import Path
import sys
import importlib
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.models as models
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import timm
PROJECT_ROOT = Path.cwd().parents[1]
COMMON_DIR = PROJECT_ROOT / "common"
DATA_ROOT = PROJECT_ROOT / "AML_project_herbarium_dataset"
LIST_DIR = DATA_ROOT / "list"

if str(COMMON_DIR) not in sys.path:
    sys.path.append(str(COMMON_DIR))

from config import DATA_ROOT, TRAIN_LIST
import data_utils
importlib.reload(data_utils)
from data_utils import HerbFieldDataset, build_test_dataset, get_with_without_label_sets , _parse_train_list
from data_utils_v2 import DualDataset, TestDataset, train_tf , val_tf, load_species_mapping
from transforms import build_eval_transform
ID2IDX = load_species_mapping(LIST_DIR)

# Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# 1. Create Train Dataset with Eval Transforms (Clean images for stable prototypes)
train_samples = _parse_train_list(TRAIN_LIST, DATA_ROOT)
# We use build_eval_transform to avoid random augmentations during prototype generation
train_ds = HerbFieldDataset(train_samples, transform=build_eval_transform(518))

# 2. Create Test Dataset
test_ds = build_test_dataset()

# 3. Get Label Sets
with_set, without_set = get_with_without_label_sets()

print(f"Train Samples: {len(train_ds)}")
print(f"Test Samples: {len(test_ds)}")
print(f"Classes with/without pairs: {len(with_set)} / {len(without_set)}")

# 4. Prepare Dual Stream train_ds and test_ds
groundtruth = {}
with open(LIST_DIR / "groundtruth.txt") as f:
    for line in f:
        path, orig_lab = line.strip().split()
        orig_lab = int(orig_lab)
        new_lab = ID2IDX[orig_lab]
        groundtruth[path] = new_lab
        
val_items = []
with open(LIST_DIR / "test.txt") as f:
    for line in f:
        path = line.strip()
        if path in groundtruth:
            val_items.append((path, groundtruth[path]))

val_ds = TestDataset(DATA_ROOT, val_items, val_tf)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=4)

paired_species = set()
with open(LIST_DIR / "class_with_pairs.txt") as f:
    for l in f:
        paired_species.add(ID2IDX[int(l.strip())])

unpaired_species = set()
with open(LIST_DIR / "class_without_pairs.txt") as f:
    for l in f:
        unpaired_species.add(ID2IDX[int(l.strip())])
# Configuration
EMBED_DIM = 512  # Dimension used in 2_stream
NUM_CLASSES = 100 # As per dataset

Train Samples: 4744
Test Samples: 207
Classes with/without pairs: 60 / 40


# Architrcture Setup

In [10]:
# --- Metric Model Architecture ---
class TripletEncoder(nn.Module):
    def __init__(self, embed_dim=512, backbone_type="dinov2_vitb14"):
        super().__init__()
        self.backbone = timm.create_model(
            "vit_base_patch14_reg4_dinov2.lvd142m",
            pretrained=False,
            num_classes=0
        )
        feat_dim = self.backbone.num_features
        self.proj_head = nn.Sequential(
            nn.Linear(feat_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, embed_dim)
        )

    def forward(self, x):
        feats = self.backbone(x)
        z = self.proj_head(feats)
        z = F.normalize(z, p=2, dim=-1)
        return z

# --- Triplet Model Architecture ---
class DinoTriplet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = timm.create_model(
            "vit_base_patch14_reg4_dinov2.lvd142m",
            pretrained=False,
            num_classes=0,
        )
        embed_dim = self.backbone.num_features
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        return self.encode(x)

    def encode(self, x, normalize=True):
        feats = self.backbone(x)
        if normalize:
            return F.normalize(feats, p=2, dim=1)
        return feats
    
# --- Dual Stream Architecture Definition ---
class ArcFace(nn.Module):
    def __init__(self, in_features, out_features, s=30, m=0.3):
        super().__init__()
        self.s = s
        self.m = m
        self.W = nn.Parameter(torch.randn(out_features, in_features))
        nn.init.xavier_uniform_(self.W)

    def forward(self, emb, labels):
        emb = F.normalize(emb, dim=1)
        W = F.normalize(self.W, dim=1)
        cosine = F.linear(emb, W)
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.unsqueeze(1), 1)
        logits = cosine - one_hot * self.m
        return logits * self.s

def create_backbone():
    # Standard init, weights will be overwritten by load_state_dict
    m = models.efficientnet_b3(weights=None) 
    m.classifier = nn.Identity()
    return m

class DualStreamArcFace(nn.Module):
    def __init__(self, n_classes=100, embed_dim=512):
        super().__init__()
        # Two separate backbones
        self.bh = create_backbone() # Herbarium Backbone
        self.bf = create_backbone() # Field Backbone

        # Projection head
        self.embed = nn.Sequential(
            nn.Linear(1536, embed_dim),
            nn.ReLU(inplace=True),
            nn.LayerNorm(embed_dim)
        )

        self.arc = ArcFace(embed_dim, n_classes)

    def forward(self, x, domain, labels=None):
        # Select backbone based on domain (0=Herb, 1=Field)
        # Note: This logic assumes the batch contains only one domain at a time
        # during inference, which is true for our standard test loop.
        
        if domain == 0:
            feat = self.bh(x)
        else:
            feat = self.bf(x)
            
        emb = self.embed(feat)

        if labels is None:
            # Return normalized embedding
            return F.normalize(emb, p=2, dim=1)

        logits = self.arc(emb, labels)
        return emb, logits

# Save and Load Prototype Function

In [11]:
# --- Saving Prototypes ---
def save_prototypes(prototypes, counts, path):
    """Saves prototypes and their counts to a file."""
    print(f"Saving prototypes to {path}...")
    torch.save({
        "prototypes": prototypes.cpu(), # Move to CPU for storage
        "counts": counts.cpu()
    }, path)
    print("Done!")

# --- Loading Prototypes ---
def load_prototypes(path):
    """Loads prototypes from file, ready for inference."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"No prototype file found at {path}")
    
    print(f"Loading prototypes from {path}...")
    data = torch.load(path, map_location=device)
    return data["prototypes"]

def load_dual_prototypes(path, device=None):
    """
    Loads prototypes from a .pt file.
    Handles both raw Tensor saves and Dictionary saves.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Prototype file not found at: {path}")
    
    # Determine device automatically if not provided
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    print(f"Loading prototypes from {path}...")
    
    # Load the file mapping to the correct device immediately
    loaded_data = torch.load(path, map_location=device, weights_only=False)

    # CASE 1: The file is just the Tensor (Result of the create_prototypes function we fixed earlier)
    if isinstance(loaded_data, torch.Tensor):
        return loaded_data.to(device)

    # CASE 2: The file is a Dictionary containing the tensor
    elif isinstance(loaded_data, dict):
        # Check common keys
        if "prototypes" in loaded_data:
            protos = loaded_data["prototypes"]
            print(f"Loaded dictionary prototypes of shape: {protos.shape}")
            return protos.to(device)
        elif "protos" in loaded_data:
            protos = loaded_data["protos"]
            return protos.to(device)
        else:
            # Fallback: check if values are tensors
            keys = list(loaded_data.keys())
            raise KeyError(f"Could not find 'prototypes' key in dictionary. Available keys: {keys}")

    else:
        raise TypeError(f"Unsupported file format. Expected Tensor or Dict, got {type(loaded_data)}")

# Create Prototype (Metric & Triple)

In [12]:


def create_and_save_prototypes(model, train_ds, save_path, batch_size=32):
    """
    Calculates prototypes from the training set (Herbarium domain only) 
    and saves them to the specified path.
    """
    model.eval()
    
    # 1. Build Prototypes from Train (Herbarium samples only)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=4)
    
    proto_sum = None
    # Note: Ensure NUM_CLASSES and device are defined in your scope or passed as args
    proto_count = torch.zeros(NUM_CLASSES, dtype=torch.long, device=device)
    
    print(f"Building Prototypes from {len(train_ds)} samples...")
    with torch.no_grad():
        for batch in tqdm(train_loader, desc="Prototypes"):
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            domains = batch["domain"].to(device)
            
            # Domain 0 is Herbarium
            mask = (domains == 0)
            if mask.sum() == 0: continue
            
            emb = model(imgs[mask])
            lbls = labels[mask]
            
            if proto_sum is None:
                proto_sum = torch.zeros(NUM_CLASSES, emb.shape[1], device=device)
                
            for e, l in zip(emb, lbls):
                proto_sum[l] += e
                proto_count[l] += 1

    # Check if we actually found herbarium samples
    if proto_sum is None:
        raise RuntimeError("No Herbarium images found in train_ds! Check dataset loading.")

    prototypes = torch.zeros_like(proto_sum)
    for c in range(NUM_CLASSES):
        if proto_count[c] > 0:
            prototypes[c] = proto_sum[c] / proto_count[c]
            prototypes[c] = F.normalize(prototypes[c], p=2, dim=-1)

    # 2. Save Prototypes
    if not save_path.parent.exists():
        save_path.parent.mkdir(parents=True, exist_ok=True)
        
    # Removed the "if exists" check so this function forces a save/update
    save_prototypes(prototypes, proto_count, save_path)
    
    print(f"Prototypes saved to {save_path}")



# Create Prototype (Dual Stream)

In [13]:
def create_prototypes(model, loader, save_path=None):
    device = next(model.parameters()).device
    model.eval()
    
    # 1. Clear GPU cache before starting
    torch.cuda.empty_cache()

    sums = {}
    counts = {}

    print("Generating prototypes...")
    
    # 2. CRITICAL: Disable gradient calculation
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            # We don't need y on GPU for the loop logic, usually
            
            # Forward pass
            emb = model(x, domain=0, labels=None)

            # 3. Move embeddings to CPU immediately to save GPU VRAM
            emb = emb.cpu()

            for f, lab in zip(emb, y):
                lab = int(lab)
                if lab not in sums:
                    sums[lab] = f.clone()
                    counts[lab] = 1
                else:
                    sums[lab] += f
                    counts[lab] += 1

    # Reconstruct the prototypes tensor
    sorted_keys = sorted(sums.keys())
    protos_list = []
    for k in sorted_keys:
        protos_list.append(sums[k] / counts[k])

    if len(protos_list) > 0:
        # Stack on CPU first
        protos = torch.stack(protos_list)
        # Normalize
        normalized_protos = F.normalize(protos, dim=1)
        
        if save_path:
            # Save the CPU tensor to disk
            torch.save(normalized_protos, save_path)
            print(f"Prototypes saved to {save_path}")
        
        # Move back to GPU only at the very end if you need to return it
        return normalized_protos.to(device)

    return None

# Evaluation for Metric Learning and Triplet

In [14]:
def evaluate_model_with_prototypes(model, test_ds, prototypes, with_set, without_set, batch_size=32):
    """
    Evaluates model using pre-loaded prototypes.
    Skips the prototype building step.
    """
    model.eval()
    prototypes = prototypes.to(device) # Ensure prototypes are on the correct device
    
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)
    stats = {k: {"correct": 0, "total": 0} for k in ["all", "with", "without"]}
    
    print("Testing with loaded prototypes...")
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            
            emb = model(imgs)
            
            # Cosine Similarity
            sims = emb @ prototypes.T
            preds = sims.argmax(dim=1)
            
            for p, l in zip(preds, labels):
                lbl_item = l.item()
                is_correct = (p == l).item()
                
                stats["all"]["total"] += 1
                if is_correct: stats["all"]["correct"] += 1
                
                if lbl_item in with_set:
                    stats["with"]["total"] += 1
                    if is_correct: stats["with"]["correct"] += 1
                elif lbl_item in without_set:
                    stats["without"]["total"] += 1
                    if is_correct: stats["without"]["correct"] += 1
    
    return {
        "Overall": stats["all"]["correct"] / stats["all"]["total"] if stats["all"]["total"] > 0 else 0,
        "With-Pair": stats["with"]["correct"] / stats["with"]["total"] if stats["with"]["total"] > 0 else 0,
        "Without-Pair": stats["without"]["correct"] / stats["without"]["total"] if stats["without"]["total"] > 0 else 0
    }

# Evaluation for Dual Stream Network

In [15]:
@torch.no_grad()
def fused_predict(model, x, domain, prototypes, alpha=0.5):
    """
    Combines ArcFace classifier logits with Prototype similarity logits.
    alpha: Weight for the prototype score (default 0.5).
    """
    x = x.to(device)
    
    # 1. Get the embedding from the specific domain backbone (0=Herb, 1=Field)
    emb = model(x, domain, labels=None)

    # 2. Prototype Similarity (Metric Learning score)
    # We use dot product (cosine sim if normalized) scaled by 30 (s parameter)
    proto_logits = emb @ prototypes.t() * 30

    # 3. ArcFace Classifier Logits (Classification score)
    # We manually calculate linear layer output using normalized W
    W = F.normalize(model.arc.W, dim=1)
    cls_logits = F.linear(F.normalize(emb, dim=1), W)

    # 4. Weighted Sum
    return cls_logits + alpha * proto_logits

def evaluate(model, dataset_or_loader, prototypes):
    model.eval()
    
    # 1. Auto-wrap in DataLoader if a Dataset is passed
    if isinstance(dataset_or_loader, torch.utils.data.DataLoader):
        loader = dataset_or_loader
    else:
        print("Wrapping dataset in DataLoader for evaluation...")
        loader = torch.utils.data.DataLoader(
            dataset_or_loader, 
            batch_size=32, 
            shuffle=False, 
            num_workers=0
        )

    total = paired_total = unpaired_total = 0
    overall_correct = paired_correct = unpaired_correct = 0

    # Move prototypes to device once
    prototypes = prototypes.to(device)

    print("Starting Evaluation...")
    
    for batch in loader:
        # --- UNIVERSAL UNPACKING LOGIC ---
        if isinstance(batch, dict):
            # Case A: Dictionary (like test_ds)
            x = batch['image']
            y = batch['label']
        elif isinstance(batch, (list, tuple)):
            # Case B: List/Tuple (like val_ds)
            # Take the first two elements, ignore filepaths/metadata if present
            x = batch[0]
            y = batch[1]
        else:
            raise TypeError(f"Unexpected batch type: {type(batch)}")
        # ---------------------------------
        
        x, y = x.to(device), y.to(device)

        # Validation is Field data, so domain=1
        with torch.no_grad():
            logits = fused_predict(model, x, 1, prototypes)
            preds = logits.argmax(1).cpu().tolist()
            
        true = y.cpu().tolist()

        for t, p in zip(true, preds):
            total += 1
            if p == t: overall_correct += 1

            # Check paired/unpaired logic
            if t in paired_species:
                paired_total += 1
                if p == t: paired_correct += 1
            else:
                unpaired_total += 1
                if p == t: unpaired_correct += 1

    return {
        "overall": overall_correct / max(total, 1) * 100,
        "paired": paired_correct / max(paired_total, 1) * 100,
        "unpaired": unpaired_correct / max(unpaired_total, 1) * 100
    }

# Running Models' Evaluation

In [ ]:
# Define paths (Adjust if your folder structure is different)
# metric checkpoint is usually in experiments/2_stream/checkpoints_k5
path_m = PROJECT_ROOT / "experiments" / "ensemble" / "metric.pt"
save_path_m = PROJECT_ROOT / "experiments" / "ensemble" / "prototype_metric.pt"
# Triplet checkpoint is usually in experiments/triplet/checkpoints (or similar)
path_tr = PROJECT_ROOT / "experiments" / "ensemble" / "triplet.pt"
save_path_tr = PROJECT_ROOT / "experiments" / "ensemble" / "prototypes_triplet.pt"
# EfficientNetB3 checkpoint is usually in experiments/efficientnetb3/checkpoints (or similar)
path_enb3 = PROJECT_ROOT / "experiments" / "ensemble" / "efficientNetB3.pt"
save_path_enb3 = PROJECT_ROOT / "experiments" / "ensemble" / "prototypes_efficientNetB3.pt"

# ===================================
# --- 1. Evaluate Metric Learning ---
# ===================================

model_metric = TripletEncoder(embed_dim=512).to(device)
state = torch.load(path_m, map_location=device)
# Handle wrapped state dicts
if 'model_state' in state: state = state['model_state']
model_metric.load_state_dict(state, strict=True)

if not save_path_m.exists():
    # Create and Save Prototypes
    create_and_save_prototypes(model_metric, train_ds, save_path_m)

# Load Prototypes
prototypes_2s = load_prototypes(save_path_m)
res_2s = evaluate_model_with_prototypes(model_metric, test_ds, prototypes_2s, with_set, without_set)
print(f"Metric Learning Results: {res_2s}")

# ===================================
# -------- 2. Evaluate Triplet ------
# ===================================

model_triplet = DinoTriplet(num_classes=NUM_CLASSES).to(device)
state = torch.load(path_tr, map_location=device)
if 'model_state' in state: state = state['model_state']
model_triplet.load_state_dict(state, strict=True)

if not save_path_tr.exists():
    # Create and Save Prototypes
    create_and_save_prototypes(model_triplet, train_ds, save_path_tr)

# Load Prototypes
prototypes_tr = load_prototypes(save_path_tr)
# Evaluate
res_tr = evaluate_model_with_prototypes(model_triplet, test_ds, prototypes_tr, with_set, without_set)
print(f"Triplet Results: {res_tr}")

# ===================================
# ----- 3. Evaluate Dual Stream -----
# ===================================
model_enb3 = DualStreamArcFace().to(device)
state = torch.load(path_enb3, map_location=device)
if 'model_state' in state: state = state['model_state']
model_enb3.load_state_dict(state, strict=True)

if not save_path_enb3.exists():
    # Create and Save Prototypes
    create_prototypes(model_enb3, val_loader, save_path_enb3)

prototype_dual = load_dual_prototypes(save_path_enb3, device=device)
# Evaluate
res_dual = evaluate(model_enb3, val_ds, prototype_dual)
print(f"DualStream Results: {res_dual}")


RuntimeError: Error(s) in loading state_dict for DualStreamArcFace:
	Missing key(s) in state_dict: "bh.features.0.0.weight", "bh.features.0.1.weight", "bh.features.0.1.bias", "bh.features.0.1.running_mean", "bh.features.0.1.running_var", "bh.features.1.0.block.0.0.weight", "bh.features.1.0.block.0.1.weight", "bh.features.1.0.block.0.1.bias", "bh.features.1.0.block.0.1.running_mean", "bh.features.1.0.block.0.1.running_var", "bh.features.1.0.block.1.fc1.weight", "bh.features.1.0.block.1.fc1.bias", "bh.features.1.0.block.1.fc2.weight", "bh.features.1.0.block.1.fc2.bias", "bh.features.1.0.block.2.0.weight", "bh.features.1.0.block.2.1.weight", "bh.features.1.0.block.2.1.bias", "bh.features.1.0.block.2.1.running_mean", "bh.features.1.0.block.2.1.running_var", "bh.features.1.1.block.0.0.weight", "bh.features.1.1.block.0.1.weight", "bh.features.1.1.block.0.1.bias", "bh.features.1.1.block.0.1.running_mean", "bh.features.1.1.block.0.1.running_var", "bh.features.1.1.block.1.fc1.weight", "bh.featu

# Hybrid Interace two combine two chosen models' performance

In [ ]:

def run_hybrid_inference_1(threshold, return_top5=False):
    """
    Hybrid Strategy:
    1. Get prediction from 'model_triplet' (The Specialist).
    2. Verify it using 'model_metric' (The Generalist) by checking the distance 
       in the 2-stream embedding space.
    3. If distance < threshold: TRUST model_triplet.
    4. If distance > threshold: REJECT model_triplet, use model_metric's nearest neighbor.
    
    If return_top5=True, also returns top-5 predictions for each sample.
    """
    hybrid_preds = []
    hybrid_top5 = []
    targets = []

    # Define the Specialist(well in paired classes) and Generalist(well in unpaired classes) prototypes
    Specialist = model_triplet.eval()
    Generalist = model_metric.eval()
    
    # We need the dataloader again
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)
    
    with torch.no_grad():
        for batch in test_loader: 
            imgs = batch["image"].to(device)
            lbls = batch["label"].to(device)
            targets.extend(lbls.cpu().numpy())
            
            # --- Step 1: Get Candidate from Specialist (model_triplet) ---
            # Note: model_triplet in your code works via prototypes (Nearest Neighbor), 
            # so we get the prediction using its embedding space.
            emb_tr = Specialist(imgs)
            sims_tr = emb_tr @ prototypes_tr.T
            pred_class_tr = sims_tr.argmax(dim=1) # The class model_triplet thinks it is
            top5_tr = sims_tr.topk(5, dim=1)[1]

            # --- Step 2: Get Embedding from Generalist (model_metric) ---
            emb_2s = Generalist(imgs)

            # --- Step 3: Distance Verification ---
            # Look up the 2-Stream prototype for the class model_triplet predicted.
            # "model_triplet says it's Class X. Is it close to Class X in the 2-stream space?"
            proposed_protos = prototypes_2s[pred_class_tr] # [Batch, 512]
            
            # Calculate Euclidean Distance
            # (Since vectors are normalized, Euclidean is safe)
            dists = torch.norm(emb_2s - proposed_protos, dim=1) # [Batch]
            
            # --- Step 4: The Decision ---
            batch_preds = []
            batch_top5 = []
            for i in range(len(imgs)):

                if dists[i].item() < threshold:
                    # PASSED: The image looks like that class in the Generalist space too.
                    # We trust the Specialist (model_triplet) because it's usually more accurate on paired data.
                    batch_preds.append(pred_class_tr[i].item())
                    if return_top5:
                        batch_top5.append(top5_tr[i].cpu().tolist())
                else:
                    # FAILED: The image is too far from that class center in 2-stream space.
                    # The Specialist is likely hallucinating (it's an Unpaired class).
                    # Fallback: Trust the Generalist (model_metric) nearest neighbor.
                    
                    # Calculate distance to ALL 2-stream prototypes
                    # emb_2s[i]: [1, 512] vs prototypes_2s: [100, 512]
                    # We use cosine similarity (dot product) to match your eval logic, 
                    # or euclidean. Since your eval used dot product argmax:
                    sims_2s = emb_2s[i].unsqueeze(0) @ prototypes_2s.T
                    nn_class = sims_2s.argmax().item()
                    batch_preds.append(nn_class)
                    if return_top5:
                        top5_2s = sims_2s.topk(5, dim=1)[1][0].cpu().tolist()
                        batch_top5.append(top5_2s)
            
            hybrid_preds.extend(batch_preds)
            if return_top5:
                hybrid_top5.extend(batch_top5)

    if return_top5:
        return hybrid_preds, targets, hybrid_top5
    return hybrid_preds, targets

In [ ]:
def run_hybrid_inference_2(threshold, return_top5=False):
    """
    Hybrid 2: Triplet Specialist + DualStream Generalist (margin-based gating).

    - Specialist  : model_triplet (DINOv2) on 518x518, prototypes_tr
    - Generalist  : model_enb3 (DualStreamArcFace) on 380x380, prototype_dual
    - Gate signal : Triplet similarity margin (top1 - top2).
        * margin >= threshold  -> trust Triplet
        * margin <  threshold  -> fall back to DualStream
    """
    # --- Models & prototypes ---
    Specialist = model_triplet
    Generalist = model_enb3

    Specialist.eval()
    Generalist.eval()

    protos_spec = prototypes_tr.to(device)    # Triplet prototypes
    protos_gen  = prototype_dual.to(device)   # DualStream prototypes

    hybrid_preds = []
    hybrid_top5  = []
    targets      = []

    from torch.utils.data import DataLoader
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)

    print(f"Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold={threshold:.3f}")

    with torch.no_grad():
        for batch in test_loader:
            # Your dataset is dict-based; keep a fallback for tuple just in case
            if isinstance(batch, dict):
                imgs = batch["image"]
                lbls = batch["label"]
            else:
                imgs, lbls = batch

            imgs = imgs.to(device)
            lbls = lbls.to(device)
            targets.extend(lbls.cpu().tolist())

            # ===== Specialist (Triplet) on 518×518 =====
            imgs_518 = F.interpolate(
                imgs,
                size=(518, 518),
                mode="bicubic",
                align_corners=False
            )

            emb_tr = Specialist(imgs_518)              # [B, D] normalized embeddings
            sims_tr = emb_tr @ protos_spec.T           # [B, C] similarity to triplet prototypes

            pred_tr = sims_tr.argmax(dim=1)            # [B]
            top5_tr = sims_tr.topk(5, dim=1)[1]        # [B, 5]

            # Triplet confidence margin: top1 - top2
            top2_vals, _ = sims_tr.topk(2, dim=1)      # [B, 2]
            margins = top2_vals[:, 0] - top2_vals[:, 1]

            # ===== Generalist (DualStream) on 380×380 =====
            # Field/test domain = 1
            emb_gen = Generalist(imgs, domain=1)       # [B, D] normalized embeddings
            sims_gen = emb_gen @ protos_gen.T          # [B, C]

            pred_gen = sims_gen.argmax(dim=1)          # [B]
            top5_gen = sims_gen.topk(5, dim=1)[1]      # [B, 5]

            # ===== Gating per-sample =====
            batch_preds = []
            batch_top5  = []

            for i in range(imgs.size(0)):
                margin_i = margins[i].item()

                if margin_i >= threshold:
                    # Triplet is confident -> trust Specialist
                    pred_i = int(pred_tr[i].item())
                    batch_preds.append(pred_i)

                    if return_top5:
                        batch_top5.append(top5_tr[i].cpu().tolist())
                else:
                    # Triplet uncertain -> fall back to DualStream
                    pred_i = int(pred_gen[i].item())
                    batch_preds.append(pred_i)

                    if return_top5:
                        batch_top5.append(top5_gen[i].cpu().tolist())

            hybrid_preds.extend(batch_preds)
            if return_top5:
                hybrid_top5.extend(batch_top5)

    if return_top5:
        return hybrid_preds, targets, hybrid_top5
    return hybrid_preds, targets


In [ ]:
# --- Tuning Hybrid Threshold ---
print("\n--- Tuning Hybrid Threshold ---")

# Range of distances to test. 
# Lower (0.3) = Strict, trusts Generalist more.
# Higher (1.2) = Loose, trusts Specialist more.
# thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
# thresholds = [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00]

thresholds = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
thresholds = [0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25]

best_acc = 0.0
best_threshold = 0.0

for th in thresholds:
    preds, tgts = run_hybrid_inference_2(th)
    correct = sum(1 for p, t in zip(preds, tgts) if p == t)
    acc = correct / len(tgts)
    print(f"Threshold {th:.2f} -> Overall Acc: {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_threshold = th

print("Best threshold:", best_threshold, "Acc:", best_acc)


--- Tuning Hybrid Threshold ---
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.000
Threshold 0.00 -> Overall Acc: 0.7681
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.050
Threshold 0.00 -> Overall Acc: 0.7681
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.050
Threshold 0.05 -> Overall Acc: 0.8261
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.100
Threshold 0.05 -> Overall Acc: 0.8261
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.100
Threshold 0.10 -> Overall Acc: 0.8454
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.150
Threshold 0.10 -> Overall Acc: 0.8454
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.150
Threshold 0.15 -> Overall Acc: 0.8551
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.200
Threshold 0.15 -> Overall Acc: 0.8551
Running

In [ ]:
# --- Final Detailed Evaluation ---
# best_threshold = 0.9
print(f"\nRunning Final Breakdown with Threshold {best_threshold}...")
final_preds, final_targets, final_top5 = run_hybrid_inference_2(best_threshold, return_top5=True)

# Manual Breakdown Calculation (Top-1 and Top-5)
stats = {
    k: {"correct1": 0, "correct5": 0, "total": 0} 
    for k in ["all", "with", "without"]
}

for p, t, top5 in zip(final_preds, final_targets, final_top5):
    is_correct1 = (p == t)
    is_correct5 = (t in top5)
    
    stats["all"]["total"] += 1
    if is_correct1: stats["all"]["correct1"] += 1
    if is_correct5: stats["all"]["correct5"] += 1
    
    if t in with_set:
        stats["with"]["total"] += 1
        if is_correct1: stats["with"]["correct1"] += 1
        if is_correct5: stats["with"]["correct5"] += 1
    elif t in without_set:
        stats["without"]["total"] += 1
        if is_correct1: stats["without"]["correct1"] += 1
        if is_correct5: stats["without"]["correct5"] += 1

results = {
    "Overall Top-1": stats["all"]["correct1"] / stats["all"]["total"],
    "Overall Top-5": stats["all"]["correct5"] / stats["all"]["total"],
    "With-Pair Top-1": stats["with"]["correct1"] / stats["with"]["total"],
    "With-Pair Top-5": stats["with"]["correct5"] / stats["with"]["total"],
    "Without-Pair Top-1": stats["without"]["correct1"] / stats["without"]["total"],
    "Without-Pair Top-5": stats["without"]["correct5"] / stats["without"]["total"]
}

print("Hybrid Results:")
for metric, value in results.items():
    print(f"  {metric}: {value:.2%}")


Running Final Breakdown with Threshold 0.15...
Running Hybrid 2 (Triplet specialist, DualStream generalist) with threshold=0.150
Hybrid Results:
  Overall Top-1: 85.51%
  Overall Top-5: 94.20%
  With-Pair Top-1: 94.12%
  With-Pair Top-5: 97.39%
  Without-Pair Top-1: 61.11%
  Without-Pair Top-5: 85.19%
Hybrid Results:
  Overall Top-1: 85.51%
  Overall Top-5: 94.20%
  With-Pair Top-1: 94.12%
  With-Pair Top-5: 97.39%
  Without-Pair Top-1: 61.11%
  Without-Pair Top-5: 85.19%
